# Variants filtering

### 0. import data

In [12]:
import os
os.getcwd()

'/titan2/UDP_SV'

In [32]:
hl.stop()

In [33]:
import hail as hl

In [34]:
hl.init(tmp_dir="/titan2/UDP_SV")

2020-07-27 09:25:16 Hail: WARN: This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2020-07-27 09:25:16 Hail: INFO: SparkUI: http://163.152.180.157:4040
Running on Apache Spark version 2.4.1
SparkUI available at http://163.152.180.157:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.49-11ae8408bad0
LOGGING: writing to /home/titan/Hail/UDP/hail-20200727-0925-0.2.49-11ae8408bad0.log


In [12]:
mt= hl.import_vcf('/titan2/UDP_SV/temp_dnv_het_3_3.vcf', reference_genome='GRCh38')

In [13]:
import gzip
with open('/titan2/UDP_SV/temp_dnv_het_3_3.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'SOURCE', 'GENE_PHENO', 'NEAREST', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'SV_overlap_AF', 'SV_overlap_PC', 'SV_overlap_name', 'gnomADg', 'gnomADg_AF']
73


### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [14]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)


## lcr(low complexity region)제외
#lcr_bed = hl.import_bed('../../Resources/lcr/LCR-hs38.bed', reference_genome = 'GRCh38')
#mt = mt.filter_rows(~hl.is_defined(lcr_bed[mt.locus]))

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
l = hl.array(['0',''])
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: l.contains(x)))

In [15]:
mt.count()

2020-07-27 09:01:08 Hail: INFO: Coerced sorted dataset
2020-07-27 09:01:08 Hail: INFO: Coerced sorted dataset


(11, 1)

---

In [16]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [17]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [18]:
tb = CSQ(tb)

In [19]:
tb.export("/titan2/UDP_SV/df_csq_dnv_het_3_3.tsv")

2020-07-27 09:01:14 Hail: INFO: Coerced sorted dataset
2020-07-27 09:01:15 Hail: INFO: Coerced sorted dataset
2020-07-27 09:01:15 Hail: INFO: Coerced sorted dataset
2020-07-27 09:01:21 Hail: INFO: merging 2 files totalling 762.3M...
2020-07-27 09:01:22 Hail: INFO: while writing:
    /titan2/UDP_SV/df_csq_dnv_het_3_3.tsv
  merge time: 1.080s


In [12]:
import os
os.getcwd()

'/titan2/UDP_SV'

In [20]:
import hail as hl

In [35]:
mt= hl.import_vcf('/titan2/UDP_SV/temp_dnv_het_3_2.vcf', reference_genome='GRCh38')

In [36]:
import gzip
with open('/titan2/UDP_SV/temp_dnv_het_3_2.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'SOURCE', 'GENE_PHENO', 'NEAREST', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'SV_overlap_AF', 'SV_overlap_PC', 'SV_overlap_name', 'gnomADg', 'gnomADg_AF']
73


### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [37]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)


## lcr(low complexity region)제외
#lcr_bed = hl.import_bed('../../Resources/lcr/LCR-hs38.bed', reference_genome = 'GRCh38')
#mt = mt.filter_rows(~hl.is_defined(lcr_bed[mt.locus]))

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
l = hl.array(['0',''])
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: l.contains(x)))

In [39]:
mt.show()

2020-07-27 09:31:19 Hail: INFO: Coerced sorted dataset
2020-07-27 09:31:19 Hail: INFO: Coerced sorted dataset
2020-07-27 09:31:20 Hail: INFO: Coerced sorted dataset


,
locus,alleles
locus<GRCh38>,array<str>
chr19:34983288,"[""T"",""<DUP:TANDEM>""]"


In [40]:
mt.count()

2020-07-27 09:31:29 Hail: INFO: Coerced sorted dataset
2020-07-27 09:31:29 Hail: INFO: Coerced sorted dataset


FatalError: SparkException: Failed to get broadcast_10_piece0 of broadcast_10

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 13, localhost, executor driver): java.io.IOException: org.apache.spark.SparkException: Failed to get broadcast_10_piece0 of broadcast_10
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1333)
	at org.apache.spark.broadcast.TorrentBroadcast.readBroadcastBlock(TorrentBroadcast.scala:207)
	at org.apache.spark.broadcast.TorrentBroadcast._value$lzycompute(TorrentBroadcast.scala:66)
	at org.apache.spark.broadcast.TorrentBroadcast._value(TorrentBroadcast.scala:66)
	at org.apache.spark.broadcast.TorrentBroadcast.getValue(TorrentBroadcast.scala:96)
	at org.apache.spark.broadcast.Broadcast.value(Broadcast.scala:70)
	at is.hail.backend.spark.SparkBroadcastValue.value(SparkBackend.scala:41)
	at is.hail.expr.ir.EmitClassBuilder$$anon$1

---

In [41]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

2020-07-27 09:32:30 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [42]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [43]:
tb = CSQ(tb)

In [44]:
tb.export("/titan2/UDP_SV/df_csq_dnv_het_3_2.tsv")

2020-07-27 09:32:39 Hail: INFO: Coerced sorted dataset
2020-07-27 09:32:39 Hail: INFO: Coerced sorted dataset
2020-07-27 17:58:26 Hail: INFO: merging 1 files totalling 2.5T...


FatalError: IOException: No space left on device

Java stack trace:
org.apache.hadoop.fs.FSError: java.io.IOException: No space left on device
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.write(RawLocalFileSystem.java:248)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.write(FSDataOutputStream.java:58)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSOutputSummer.writeChunk(ChecksumFileSystem.java:424)
	at org.apache.hadoop.fs.FSOutputSummer.writeChecksumChunks(FSOutputSummer.java:206)
	at org.apache.hadoop.fs.FSOutputSummer.flushBuffer(FSOutputSummer.java:163)
	at org.apache.hadoop.fs.FSOutputSummer.flush(FSOutputSummer.java:182)
	at java.io.FilterOutputStream.flush(FilterOutputStream.java:140)
	at java.io.DataOutputStream.flush(DataOutputStre

In [ ]:
import hail as hl
mt= hl.import_vcf('/titan2/UDP_SV/temp_dnv_het_8.vcf', reference_genome='GRCh38')

In [ ]:
import gzip
with open('/titan2/UDP_SV/temp_dnv_het_8.vcf', 'rt') as f:
    for l in f:
        if 'ID=CSQ' in l:
            temp = l.strip('\n').split('|')
            break

temp[0] = 'Allele';temp[72] = 'gnomADg_AF'

print(temp)
print(len(temp))
#print(temp.index('Consequence'))

### 1. Variant filtering
> - `filter == PASS`인 variant만 남기기
> - multi allelic 확인 및 제외
> - LCR(low complexity region) 제외
> - gnomad filtering
> - ~AC ==1/2(hom)~

In [ ]:
##  `filter == PASS`인 variant만 남기기 
mt = mt.filter_rows(hl.len(mt.filters) == 0)

## multi allelic 제외
mt = hl.split_multi(mt)
mt = mt.filter_rows(mt.was_split == False)


## lcr(low complexity region)제외
#lcr_bed = hl.import_bed('../../Resources/lcr/LCR-hs38.bed', reference_genome = 'GRCh38')
#mt = mt.filter_rows(~hl.is_defined(lcr_bed[mt.locus]))

## gnomad filtering
mt = mt.annotate_rows(csq = mt.info.CSQ)
mt = mt.transmute_rows(csq_gnomADg_AF = mt.csq.map(lambda x: x.split('\|')[72]))
l = hl.array(['0',''])
mt = mt.filter_rows(mt.csq_gnomADg_AF.all(lambda x: l.contains(x)))

In [ ]:
mt.count()

---

In [ ]:
tb = mt.entries()
tb = tb.key_by(tb.locus,tb.alleles,tb.s)

### 3. HQ het filtering

---

### 4. Export data + CSQ annotation


(CSQ 전체)

In [ ]:
def CSQ(table):
    t = table
    t = t.annotate(v1 = hl.tuple([t.locus.contig.replace("chr", ""),hl.str(t.locus.position)]),
                   v2 = hl.tuple([t.alleles[0],t.alleles[1]]))
    t = t.transmute(variant = hl.delimit(hl.array([t.v1[0],t.v1[1],t.v2[0],t.v2[1]]), ":"))
    t = t.annotate(CSQ= t.info.CSQ).explode('CSQ')
    t = t.transmute(csq = t.CSQ.split('\|')).explode('csq')
    return t

In [ ]:
tb = CSQ(tb)

In [ ]:
tb.export("/titan2/UDP_SV/df_csq_dnv_het_8.tsv")